# Wstęp

Na tych zajęciach stworzysz pierwszą sieć wielowarstwową i zapoznasz się z podstawami zadania klasyfikacji obrazów. Dodatkowo zostanie przedstawione narzędzi do wizualizacji procesu uczenia. 

## Cel ćwiczenia

Celem ćwiczenia jest zapoznanie z:
*   Wielowarstwowymi sieciami neuronowymi (MLP)
*   Podstawami przetwarzania obrazów (ang. Computer Vision)
*   Metodami reguralyzacji sieci neuronowych
*   Techniką wczesnego zatrzymania uczenia
*   Tensorboardem 

## Warunki zaliczenia

W celu zaliczenia ćwiczeń należy uzupełnić wszystkie brakujące elementu kodu, wykonać wszystkie polecenia i wyuczyć opdowiednie warianty modelu.


# Zbiór danych do klasyfikacji obrazów

W ramach zadania klasyfikacji obrazów wykorzystamy zbiór [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist). Zbiór danych składa się z 70.000 obrazów o wymiarach 28x28 pikseli przedstawiających różne typy odzieży w skali szarości. Podobnie jak [MNIST](https://en.wikipedia.org/wiki/MNIST_database), w zbiorze występuje 10 klas.  

### Dane w środowisku Google Colab

Uruchomienie notebooka z wykorzystaniem Google Colab powaduje za każdym razem pobranie zbioru danych od nowa. W celu zapisania danych na "stałe" możemy wykorzystać Dysk Google, co przyda się nie tylko w kontekście pobrania zbioru, ale również przy zapisywaniu logów.

Podłączenie Dysku Google wykonuje się z wykorzystaniem funkcji `google.colab.drive.mount`
```python
from google.colab import drive
drive.mount("/content/drive/")
```
Po wskazaniu ścieżki pod jaką będzie podpięty dysk i wykonaniu komendy, zostaje nam wygenerowany link, gdzie przekazujemy uprawnienia dostępu do danych z dysku dla tego notebooka. Po zaakceptowaniu zostanie nam wygenerowany token, który wklejamy w miejscu wykonania komendy.

### Wczytanie danych
Do wczytania zbioru danych możemy wykorzystać [`torchvision`](https://pytorch.org/vision/stable/index.html), który zawiara popularne zbiory danych, modele i transformacje danych z dziedziny wizji komputerowej (ang. *computer vision*). 

Oryginalne dane są zapisane w postaci liczb całkowitych z zakresu 0-255 w postaci [obrazu PIL](https://pillow.readthedocs.io/en/stable/reference/Image.html). Do przeskalowania danych wykorzystamy transformację [`torchvision.transforms.ToTensor`](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor). Transformacje danych możemy przekazywać do loadera zbioru danych z `torchvision`. Można przekazać zarówno pojedynczą operację transformacji lub złożyć kilka operacji za pomocą [`torchvision.transforms.Compose`](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Compose)

**Uwaga** 
Operacje transform są realizowane w "locie" w momencie wywołania `__getitem__`.

In [ ]:
import os

from torchvision.datasets import FashionMNIST 
from torchvision import transforms
from google.colab import drive

drive.mount("/content/drive/")

path = '/content/drive/MyDrive/lab04/FashionMNIST/'
os.makedirs(path, exist_ok=True)

train_data = FashionMNIST(
    root=path, 
    train=True,
    download=True,
    transform=transforms.ToTensor()
)
test_data = FashionMNIST(
    root=path, 
    train=False,
    download=True,
    transform=transforms.ToTensor()
)


# Wizualizacja danych

W odróżnieniu od poprzedniej listy zadań, gdzie ręcznie rysowaliśmy krzywe uczenia i metryki, tym razem wykorzystamy narzędzie do wizualizacji [`tensorboard`](https://www.tensorflow.org/tensorboard). O ile oryginalnie Tensorboard był rozwijany dla Tensorflowa, to i tak możemy wykorzystać w PyTorchu i środowiskiem Google Colab. Funkcje pomocnicze znajdują się w module [`torch.utils.tensorboard`](https://pytorch.org/docs/stable/tensorboard.html). 

Tensorboard bazuje na logach, które umieszczamy w odpowiednim folderze. Dlatego najpierw zaczniemy od stworzenia takich logów, a dopiero w późniejszym etapie uruchomimy tensorboarda. Zapisywanie logów odbywa się za pomocą klasy [`SummaryWriter`](https://pytorch.org/docs/stable/tensorboard.html?highlight=summarywriter#torch.utils.tensorboard.writer.SummaryWriter).

In [ ]:
from torch.utils.tensorboard import SummaryWriter

log_dir = '/content/drive/MyDrive/lab04/logs/'
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir)

Wyświetlmy teraz kilka obrazów ze zbioru uczącego. Do wyświetlenia siatki zdjęć wykorzystamy funkcję pomocniczą [`torchvision.utils.make_grid`](https://pytorch.org/vision/stable/utils.html#torchvision.utils.make_grid). Do elementów zbioru uczącego, bez nałożonej transformacji, możemy się odwołać za pomocą obiektu `data`. Metoda `make_grid` oczekuje danych w formacie `Batch x Liczba Kanałów x Wysokość x Szerokość`, dlatego musimy dostosować odpowiednio wejście. Pełny kod zdefiniowano poniżej. Następnie przekazujemy utworzony w taki sposób grid, do utworzonej wcześniej instancji klasy `SummaryWriter` za pomocą metody `add_image`. 

In [ ]:
import torchvision

images_to_plot = 64
img_grid = torchvision.utils.make_grid(
    train_data.data[0:images_to_plot].reshape(images_to_plot, 1, 28, 28)
)

writer.add_image('Train data sample', img_grid)

Logi nie są już w tym momencie puste, a więc czas na uruchomienie Tensorboarda. Tensorboard w środowisku Google Colab uruchamiamy za pomocą komend magicznych `%load_ext tensorboard` i `% tensorboard --logdir logs`. 
W przypadku otrzymania błędu 403 należy również zezwolić przeglądarce na  pliki cookie osób trzecich (ang. *third party cookies*).

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $log_dir

# Implementacja architektury MLP

Teraz przejdziemy do implementacji architektury wielowarstwowego perceptrona (ang. *multilayer perceptron*, *MLP*). 
Do zdefiniowania architektury wykorzystaj moduł `torch.nn.Sequential`, przekazując obiekt `OrderDict` zawierający nazwę i warstwy. 

Przykład
```python
nn.Sequential(OrderedDict([
    ('dense1', nn.Linear(20, 10)),
    ('relu1', nn.ReLU()),
]))
```

***Zaimplementuj*** podaną architekturę sieci neuronowej

| Nazwa warstwy | Opis |
| --- | --- |
| flatten | Spłaszczenie obrazu z wymiaru **28x28** na wymiar **784** |
| dense1 | Warstwa w pełni połączona z **256** neuronami |
| relu1 | Funkcja aktywacji **ReLU** |
| dense2 | Warstwa w pełni połączona z **128** neuronami |
| relu2 | Funkcja aktywacji **ReLU** |
| dense3 | Warstwa w pełni połączona z **10** neuronami |


*Metoda inicjalizacji wag i biasów*: domyślna dla warstwy `torch.nn.Linear` (Rozkład jednostajny z zakresem wartości $\mathcal{U}(-\sqrt{k}, \sqrt{k})$, gdzie $k = \frac{1}{\text{in_features}}$)



In [ ]:
from collections import OrderedDict

from torch import nn

class MLP(nn.Module): 
  def __init__(self):
    super().__init__()

    self.net = nn.Sequential(
        ____
    )
  
  def forward(self, x: torch.Tensor):
      ____

Mając zdefiniowaną architekturę i wczytany zbiór danych możemy teraz wyświetlić naszą architekturę w postaci grafu obliczeniowego w Tensorboardzie. W tym celu wykorzystamy poniższy kod.

In [ ]:
mlp = MLP()

writer.add_graph(mlp, input_to_model=train_data[0][0])
writer.close()

Podobnie jak w liście 2., wykorzystany będzie podział zbioru na trzy części: treningowy, walidacyjny (dev) i testowy. Z tą różnicą, że zbiór testowy pozostawimy bez zmian, aby mieć możliwość porównania się do wyników [innych klasyfikatorow](http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/).   

***Zaprogramuj*** nowy podział danych:
-  Zbiór treningowy podziel na dwie cześci:
   - zbiór treninowy/uczący zawierający 54.000 elementy,
   - zbiór walidacyjny zawierajcy 6.000 elementów.
- Zbiór testowy ma pozostać w oryginalnej formie.
- Zachowaj oryginalne proporcje klas w nowoutworzonych zbiorach, tj. dokonaj stratyfikacji. W tym celu możesz wykorzystać metodę `train_test_split` z poprzednich zajęć. Tym razem operację wykonaj na indeksach i utwórz podzbiory używając klasy `torch.utils.data.Subset`. Ustaw `random_state` podziału na $1$.
- Utwórz istancje klasy `DataLoader` dla wszystkich części zbiorów. Ustaw wartość parametru `batch_size` na $128$

In [ ]:
from typing import Tuple

import numpy as np
from torch.utils.data import  DataLoader, Subset
from sklearn.model_selection import train_test_split

def split_train_data(
    train_data: torchvision.datasets.FashionMNIST
) -> Tuple[Subset, Subset]:
    train_idx, dev_idx = train_test_split(
        ____,
        test_size=___, stratify=____,
        random_state=____
    )
    train = Subset(____)
    dev = Subset(____)
    return train, dev

train, dev = split_train_data(train_data)

train_loader = DataLoader(____)
dev_loader = DataLoader(____)
test_loader = DataLoader(____)

## Inicjalizacja parametrów sieci
Domyślne parametry (w tym metoda incjalizacji wag / biasów) w bibliotece `PyTorch` są dobrane tak, aby dla różnych zadań i architektur dawały optymalne wyniki. W niektórych przypadkach dobranie odpowiedniej metody inicjalizacji wag i biasów do zastosowanej architektury, może usprawnić proces uczenia, przez co poprawić osiągi modelu. Poniżej przedstawiono przykład zastąpienia domyślnej metody inicjalizacji na inicjalizację parametrów z rozkładu normalnego dla pojedynczej warstwy.

```python
from torch import nn

layer = nn.Linear(100, 10)
nn.init.normal_(layer.weight, mean=0.0, std=1.0)
```

Alternatywnie, możemy zmienić inicjalizacje dla kilku warstw, wykorzystując metodę `apply`. 
```python
net = nn.Sequential(
      nn.Linear(100, 10),
      nn.Linear(10, 1)
)

def init_layer_params(layer: nn.modules.Module):
  if isinstance(layer, nn.Linear):
    nn.init.normal_(layer.weight, mean=0.0, std=1.0)
    nn.init.normal_(layer.bias, mean=0.0, std=1.0)

net.apply(init_layer_params)


## Generowanie wykresów w Tensorboardzie

Logowanie metryk do Tensorboarda odbywa się z wykorzystaniem metody `writer.add_scalar`. Wartości dodajemy pojedynczo. Poniżej przedstawiono przekazanie wartości funkcji straty z kolejnych epok do writera.
```python
train_loss_values = [0.78, 0.65, 0.5]
for epoch_id, train_loss in enumerate(train_loss_values):
  writer.add_scalar(
    tag='training loss', 
    scalar_value=train_loss, 
    global_step=epoch_id+1
  )
```
Nie musimy przekazać wartości osobno dla train i test można umieścić je w jednym obiekcie. Odbywa się to za pomocą metody `writer.add_scalars`. Przykładowe wywołanie dla wartości funkcji straty ze zbioru treningowego i walidacyjnego.
```python
train_loss_values = [0.78, 0.65, 0.5]
dev_loss_values = [0.96, 0.78, 0.79]

for epoch_id, (train_loss, dev_loss) in (
  enumerate(zip(train_loss_values, dev_loss_values))
):
    print(train_loss, dev_loss)
    writer.add_scalars(
      main_tag='loss', 
      tag_scalar_dict={
        'train': train_loss,
        'dev': dev_loss
      }, 
      global_step=epoch_id+1
    )
```

Wykresy do Tensorboard dodawane są za pomocą metody `add_figure`. Poniżej zaprezentowano przykład dodania macierzy pomyłek do Tensorboarda.

```python
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(10, 5))
cm = confusion_matrix(
    y_true=[0, 1, 0, 0], 
    y_pred=[1, 1, 1, 0]
)
sns.heatmap(cm, annot=True, figure=fig)
writer.add_figure(tag='Confusion matrix', figure=fig)
```


Wykorzystując funkcję `fit` z poprzednich zajęć ***dodaj***:
- Generowanie wykresów krzywej uczenia dla Tensorboarda w zależności od epoki dla zbioru uczącego oraz walidacyjnego
- Generowanie wykresów dokładności (ang. *accuracy*) dla Tensorboarda w zależności od epoki dla zbioru uczącego oraz walidacyjnego
- Macierz pomyłek dla zbioru testowego po zakończeniu uczenia

***Przeprowadź uczenie*** modelu sieci wykorzystyjąc następujące hiperparametry uczenia: 

- Funkcja straty: ***Entropia krzyżowa***
- Wielkość paczki (*min-batch*): ***128***  
- Optymalizator: ***Adam*** 
- Współczynnik uczenia: ***0.01***  
- Liczba epok: ***50*** 

***Dokonaj ewaluacji*** modelu po zakończeniu uczenia na zbiorze testowym.

In [ ]:
# UMIEŚĆ KOD 

# Wczesne zatrzymowanie uczenia (ang. *early stopping*)

Jak można zauważyć jakość klasyfikacji na zbiorze walidacyjnym po kilkunastu epokach zaczyna oscylować w takim samym zakresie wartości. Możemy wykorzystać technikę wczesnego zatrzymania, w momencie kiedy funkcja straty na zbiorze walidacyjnym przestaje maleć, zatrzymywany jest cały proces. Liczba epok po których nie dochodzi do poprawy wartości funkcji kosztu, i zatrzymywany jest proces uczenia, jest kontrolowany hiperparametrem cierpliwości (ang. *patience*).

Do zaimplementowania tej techniki potrzebne nam będzie zapisywanie modelu po każdej epoce w której doszło do poprawy jakości modelu. W tym celu używamy metody `torch.save`, który zapisuje zadany mu obiekt w postaci pythonowego pickla. W bibliotece PyTorch dostęp do wyuczalnych parametry mamy za pomocą `state_dict`. `state_dict` jest to pythonowy słownik, który mapuje każdą warstwę do tensora jej parametrów. Jeżeli chcemy użyć modelu do inferencji wystarczy, że zapiszemy wyłącznie parametry modelu. W przypadku kiedy chcemy mieć możliwość douczenia modelu w późniejszym momencie musimy również zapisać `state_dict` z wykorzystanego optymalizatora. Do nas należy decyzja co chcemy zapisać, więc oprócz samych parametrów warto zapisać numer epoki, wartości funkcji kosztu, hiperparametry modelu czy optymalizatora.

Przykładowy kod do zapisywania modelu
```python
torch.save(
    obj={
      'epoch': epoch,
      'loss': loss,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'model_args': model_args,
      'optim_args': optim_args
    },
    f=output_path
)
```

 Wczytanie modelu składa się z kilku kroków:
 - Wczytujemy zapisany przez nas punkt kontrolny (ang. *checkpoint*)
 - Inicjalizujemy model i optymalizator od nowa.
 - Ładujemy obiekt `state_dict` odpowiednio do modelu i optymalizatora

Przykładowy kod do wczytania modelu.
```python
checkpoint = torch.load(output_path)

model = ModelCls(**checkpoint['model_args'])
optimizer = OptimazerCls(**checkpoint['optim_args'])

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

```

**Zaimplementuj technikę wczesnego zatrzymania** wraz z zapisywaniem punktów kontrolnych modelu

***Zastosuj*** dwie wybrane techniki reguralyzacji sieci (L1, L2, Dropout). Sprawdź czy poprawiły one wyniki Twojego modelu na zbiorze testowym. Dodaj wykresy do Tensorboarda, dokonaj porównania i podsumowania wyników

**UWAGA** W przypadku zastosowania metody optymalizacji `Adam` i regularyzacji `L2` należy zastosować optymalizator `AdamW`. Więcej szczegółów można znaleźć w publikacji autorów optymalizatora `AdamW` https://arxiv.org/abs/1711.05101

In [ ]:
# UMIEŚĆ KOD 